In [151]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob

from sklearn import datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [152]:
data_dir = "-6dB_fan/fan/id_00/normal"
normal_audio_files = glob(data_dir + '/*.wav')
x = len(norm_audio_files)
x

1011

In [153]:
data_dir = "-6dB_fan/fan/id_00/abnormal"
abnormal_audio_files = glob(data_dir + '/*.wav')
y = len(abnormal_audio_files)
y


407

In [154]:
def zero_crossing_rate(file_name):
    data, sample_rate = librosa.load(file_name)
    zero_crossings = librosa.zero_crossings(data[:1000], pad=False)
    return zero_crossings.mean()



In [155]:
def rmse(file_name):
    data, sample_rate = librosa.load(file_name)
    rms_audio = librosa.feature.rms(data)[0]
    return rms_audio


In [156]:
def spectral_bandwidth(file_name):
    data, sample_rate = librosa.load(file_name)
    spectral_bandwidth =  librosa.feature.spectral_bandwidth(data, sr=sample_rate)[0]
    return spectral_bandwidth



In [157]:
def fourier_tempogram(file_name):
    data, sample_rate = librosa.load(file_name)
    fourier_tempogram =  librosa.feature.fourier_tempogram(data, sr=sample_rate)[0]
    return fourier_tempogram

In [158]:
def spec_rolloff(file_name):
    data, sample_rate = librosa.load(file_name)
    spectral_rolloff = librosa.feature.spectral_rolloff(data+0.01, sr=sample_rate)[0]
    return  spectral_rolloff


In [107]:
extracted_feature = []
for file in tqdm(normal_audio_files):
    file_name = file
    class_label = 0 # 0 for normal.
    zcr = zero_crossing_rate(file_name)
    
    spec_band= spectral_bandwidth(file_name)
    spec_band_min = spec_band.min()
    spec_band_max = spec_band.max()
    spec_band_mean = spec_band.mean()
    spec_band_std = spec_band.std()
    
    
    rmse_mean = rmse(file_name).mean()
    
    extracted_feature.append([file_name, zcr, spec_band_max, spec_band_min, spec_band_mean, spec_band_std,
                              rmse_mean, class_label])

100%|██████████████████████████████████████████████████████████████████████████████| 1011/1011 [10:58<00:00,  1.53it/s]


1871

In [162]:
spectrum_rolloff_list = []


In [163]:
for file in tqdm(normal_audio_files):
    file_name = file
    
    spec_rolloff_ =  spec_rolloff(file_name)
    spec_rolloff_sum = spec_rolloff_.sum()
    
    
    
    spectrum_rolloff_list.append(spec_rolloff_sum)

100%|██████████████████████████████████████████████████████████████████████████████| 1011/1011 [03:52<00:00,  4.35it/s]


In [164]:
for file in tqdm(abnormal_audio_files):
    file_name = file
    
    spec_rolloff_ =  spec_rolloff(file_name)
    spec_rolloff_sum = spec_rolloff_.sum()
    
    
    
    spectrum_rolloff_list.append(spec_rolloff_sum)

100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [01:38<00:00,  4.13it/s]


In [111]:
for file in tqdm(abnormal_audio_files):
    file_name = file
    class_label = 1 # 0 for normal.
    zcr = zero_crossing_rate(file_name)
    
    spec_band= spectral_bandwidth(file_name)
    spec_band_min = spec_band.min()
    spec_band_max = spec_band.max()
    spec_band_mean = spec_band.mean()
    spec_band_std = spec_band.std()
    
    
    rmse_mean = rmse(file_name).mean()
    
    extracted_feature.append([file_name, zcr, spec_band_max, spec_band_min, spec_band_mean, spec_band_std,
                              rmse_mean, class_label])

100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [04:31<00:00,  1.50it/s]


In [112]:
extracted_features_df = pd.DataFrame(extracted_feature,columns=['file_name', 'zcr', 'spec_band_max', 'spec_band_min',
                                                                'spec_band_mean', 'spec_band_std', 
                                                                'rmse_mean', 'class'])

In [126]:
extracted_features_df

,file_name,zcr,spec_band_max,spec_band_min,spec_band_mean,spec_band_std,rmse_mean,class
0,-6dB_fan/fan/id_00/normal\00000000.wav,0.033,2186.859945,1199.106924,1283.824395,56.193976,0.010253,0
1,-6dB_fan/fan/id_00/normal\00000001.wav,0.074,1964.498781,1601.458971,1749.899728,46.057723,0.009026,0
2,-6dB_fan/fan/id_00/normal\00000002.wav,0.044,1611.333345,1322.125918,1417.241976,31.888845,0.010177,0
3,-6dB_fan/fan/id_00/normal\00000003.wav,0.090,1903.834562,1646.409253,1782.300090,47.540587,0.008910,0
4,-6dB_fan/fan/id_00/normal\00000004.wav,0.026,1859.185695,1245.459978,1394.553652,89.329761,0.010531,0
...,...,...,...,...,...,...,...,...
1413,-6dB_fan/fan/id_00/abnormal\00000402.wav,0.060,1991.972897,1016.851359,1368.177800,188.335089,0.009734,1
1414,-6dB_fan/fan/id_00/abnormal\00000403.wav,0.070,2016.924958,1288.197948,1534.996473,116.475586,0.010159,1
1415,-6dB_fan/fan/id_00/abnormal\00000404.wav,0.038,2010.045068,1247.780494,1346.694567,46.210524,0.010522,1
1416,-6dB_fan/fan/id_00/abnormal\00000405.wav,0.029,1917.085187,1162.319349,1318.473823,122.268276,0.010673,1


In [170]:

extracted_features_df['spectrum_rolloff_l'] = spec_band_sum_list
extracted_features_df

,file_name,zcr,spec_band_sum,spec_band_max,spec_band_min,spec_band_mean,spec_band_std,rmse_mean,class,spectrum_rolloff_l
0,-6dB_fan/fan/id_00/normal\00000000.wav,0.033,553328.314402,2186.859945,1199.106924,1283.824395,56.193976,0.010253,0,553328.314402
1,-6dB_fan/fan/id_00/normal\00000001.wav,0.074,754206.782847,1964.498781,1601.458971,1749.899728,46.057723,0.009026,0,754206.782847
2,-6dB_fan/fan/id_00/normal\00000002.wav,0.044,610831.291691,1611.333345,1322.125918,1417.241976,31.888845,0.010177,0,610831.291691
3,-6dB_fan/fan/id_00/normal\00000003.wav,0.090,768171.338788,1903.834562,1646.409253,1782.300090,47.540587,0.008910,0,768171.338788
4,-6dB_fan/fan/id_00/normal\00000004.wav,0.026,601052.624223,1859.185695,1245.459978,1394.553652,89.329761,0.010531,0,601052.624223
...,...,...,...,...,...,...,...,...,...,...
1413,-6dB_fan/fan/id_00/abnormal\00000402.wav,0.060,589684.631613,1991.972897,1016.851359,1368.177800,188.335089,0.009734,1,589684.631613
1414,-6dB_fan/fan/id_00/abnormal\00000403.wav,0.070,661583.479851,2016.924958,1288.197948,1534.996473,116.475586,0.010159,1,661583.479851
1415,-6dB_fan/fan/id_00/abnormal\00000404.wav,0.038,580425.358225,2010.045068,1247.780494,1346.694567,46.210524,0.010522,1,580425.358225
1416,-6dB_fan/fan/id_00/abnormal\00000405.wav,0.029,568262.217837,1917.085187,1162.319349,1318.473823,122.268276,0.010673,1,568262.217837


In [171]:
new_cols =['file_name','spectrum_rolloff_l', 'zcr','spec_band_sum', 'spec_band_max', 'spec_band_min','spec_band_mean', 'spec_band_std','rmse_mean', 'class']
extracted_features_df=extracted_features_df[new_cols]
extracted_features_df

,file_name,spectrum_rolloff_l,zcr,spec_band_sum,spec_band_max,spec_band_min,spec_band_mean,spec_band_std,rmse_mean,class
0,-6dB_fan/fan/id_00/normal\00000000.wav,553328.314402,0.033,553328.314402,2186.859945,1199.106924,1283.824395,56.193976,0.010253,0
1,-6dB_fan/fan/id_00/normal\00000001.wav,754206.782847,0.074,754206.782847,1964.498781,1601.458971,1749.899728,46.057723,0.009026,0
2,-6dB_fan/fan/id_00/normal\00000002.wav,610831.291691,0.044,610831.291691,1611.333345,1322.125918,1417.241976,31.888845,0.010177,0
3,-6dB_fan/fan/id_00/normal\00000003.wav,768171.338788,0.090,768171.338788,1903.834562,1646.409253,1782.300090,47.540587,0.008910,0
4,-6dB_fan/fan/id_00/normal\00000004.wav,601052.624223,0.026,601052.624223,1859.185695,1245.459978,1394.553652,89.329761,0.010531,0
...,...,...,...,...,...,...,...,...,...,...
1413,-6dB_fan/fan/id_00/abnormal\00000402.wav,589684.631613,0.060,589684.631613,1991.972897,1016.851359,1368.177800,188.335089,0.009734,1
1414,-6dB_fan/fan/id_00/abnormal\00000403.wav,661583.479851,0.070,661583.479851,2016.924958,1288.197948,1534.996473,116.475586,0.010159,1
1415,-6dB_fan/fan/id_00/abnormal\00000404.wav,580425.358225,0.038,580425.358225,2010.045068,1247.780494,1346.694567,46.210524,0.010522,1
1416,-6dB_fan/fan/id_00/abnormal\00000405.wav,568262.217837,0.029,568262.217837,1917.085187,1162.319349,1318.473823,122.268276,0.010673,1


In [140]:
extracted_features_df.to_csv('csv_files/fan_id_00_normal.csv',index=False)

In [172]:
def filter_features(df,y,treshold=0.10):
    relevant_features_indx = []
    for i in range(len(df.columns)):
        print(f"{df.columns[i]}: {np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]}, threshold:{treshold}, boolean:{abs(np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]) >treshold}")
        if abs(np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]) >treshold:
            relevant_features_indx.append(i)
    return relevant_features_indx

f_cols = filter_features(extracted_features_df.iloc[:,1:-1],extracted_features_df.iloc[:,-1].values,treshold=0.10)

spectrum_rolloff_l: -0.06542224939287414, threshold:0.1, boolean:False
zcr: -0.10930470303878594, threshold:0.1, boolean:True
spec_band_sum: -0.06542224939287414, threshold:0.1, boolean:False
spec_band_max: -0.01146899120094138, threshold:0.1, boolean:False
spec_band_min: -0.07141624528573425, threshold:0.1, boolean:False
spec_band_mean: -0.06542224939287417, threshold:0.1, boolean:False
spec_band_std: 0.020540837931132618, threshold:0.1, boolean:False
rmse_mean: 0.4354967426676065, threshold:0.1, boolean:True


In [174]:
X_train, X_test, y_train, y_test = train_test_split(extracted_features_df.iloc[:,1:-1].values, extracted_features_df.iloc[:,-1].values, test_size=0.3, random_state=42)

In [175]:
pipeline_lr = Pipeline([('scalar1',StandardScaler()),
                        ('lr_classifier',LogisticRegression(random_state=0))])

In [177]:
pipeline_rf = Pipeline([('scalar2',StandardScaler()),
                        ('rf_classifier',RandomForestClassifier(n_estimators=int((len(extracted_features_df.columns)**0.5//1)), criterion= 'entropy',random_state=0))])

In [178]:
pipeline_dt = Pipeline([('scalar3',StandardScaler()),
                        ('dt_classifier',DecisionTreeClassifier(criterion="entropy",random_state=0))])

In [179]:
pipeline_GaussianNB = Pipeline([('scalar4',StandardScaler()),
                        ('dt_classifier',GaussianNB())])

In [180]:
pipelines = [pipeline_lr,pipeline_rf,pipeline_dt,pipeline_GaussianNB]

In [181]:
pipe_dict = {0:'Logistic Regression',1:'Random Forest',2:'Decision Tree',3:'GaussianNB'}

In [182]:
for pipe in pipelines:
    pipe.fit(X_train,y_train)

In [184]:
best_accuracy=0.0
best_classifier_indx = 0
best_pipeline=""

In [185]:
for i,model in enumerate(pipelines):
    print(f"{pipe_dict[i]} test accuracy: {model.score(X_test,y_test)} ")
    if model.score(X_test,y_test)>best_accuracy:
        best_accuracy=  model.score(X_test,y_test)
        best_pipeline=model
        best_classifier_idnx = i
print(f"Classifier with the best accuracy:{pipe_dict[best_classifier_idnx]}")

Logistic Regression test accuracy: 0.8286384976525821 
Random Forest test accuracy: 0.7629107981220657 
Decision Tree test accuracy: 0.7511737089201878 
GaussianNB test accuracy: 0.8286384976525821 
Classifier with the best accuracy:Logistic Regression


In [190]:
pipelines[0].fit(X_train,y_train)
pipelines[0].score(X_test,y_test)


0.8286384976525821

In [193]:
y

407